# Laboratorio 6
## Threat Hunting
Búsqueda y caza de amenazas dentro de un tráfico de red
> #### Gerardo Méndez 18239

In [3]:
# math
import numpy as np
import pandas as pd
import seaborn as sns

# flare
import flare as fl

# utils
import json
import random

# graphs
sns.set(color_codes=True)
%matplotlib inline

## PARTE 1
Filtrado y Preprocesamiento

### Carga de JSONs

In [4]:
eva = []

with open('data/large.json', 'r') as file:
    while (line := file.readline().rstrip()):
        json_data = json.loads(line)
        eva.append(json_data)

In [5]:
eva

[{'timestamp': '2017-07-22T17:33:15.784100-0500',
  'flow_id': 283570730437312,
  'pcap_cnt': 18519,
  'event_type': 'alert',
  'vlan': 130,
  'src_ip': '192.168.203.200',
  'src_port': 2328,
  'dest_ip': '192.181.145.109',
  'dest_port': 445,
  'proto': 'TCP',
  'alert': {'action': 'allowed',
   'gid': 1,
   'signature_id': 2001569,
   'rev': 15,
   'signature': 'ET SCAN Behavioral Unusual Port 445 traffic Potential Scan or Infection',
   'category': 'Misc activity',
   'severity': 3}},
 {'timestamp': '2017-07-22T17:33:16.571273-0500',
  'flow_id': 1519048138171115,
  'pcap_cnt': 21531,
  'event_type': 'ssh',
  'vlan': 140,
  'src_ip': '192.168.204.60',
  'src_port': 62990,
  'dest_ip': '192.168.26.254',
  'dest_port': 22,
  'proto': 'TCP',
  'ssh': {'client': {'proto_version': '2.0',
    'software_version': 'OpenSSH_5.0'},
   'server': {'proto_version': '1.99', 'software_version': 'Cisco-1.25'}}},
 {'timestamp': '2017-07-22T17:33:16.661646-0500',
  'flow_id': 1327836194150542,
  'pca

In [6]:
print('Número de registros totales: ', len(eva))

Número de registros totales:  746909


### Filtrar registros DNS

In [7]:
clean = [reg for reg in eva if 'dns' in list(reg.keys())]

##### Cantidad de registros

In [8]:
print('Número de registros filtrados: ', len(clean))

Número de registros filtrados:  15749


##### Sample de registros

In [9]:
random.choice(clean)

{'timestamp': '2017-07-22T19:29:18.790132-0500',
 'flow_id': 929102624066938,
 'pcap_cnt': 3354344,
 'event_type': 'dns',
 'vlan': 150,
 'src_ip': '192.168.207.4',
 'src_port': 53,
 'dest_ip': '192.168.205.188',
 'dest_port': 53670,
 'proto': 'UDP',
 'dns': {'type': 'answer',
  'id': 58783,
  'rcode': 'NXDOMAIN',
  'rrname': 'mirrors.xmission.com'}}

In [10]:
random.choice(clean)

{'timestamp': '2017-07-22T19:00:26.140717-0500',
 'flow_id': 2121887943108013,
 'pcap_cnt': 2280784,
 'event_type': 'dns',
 'vlan': 150,
 'src_ip': '192.168.205.188',
 'src_port': 46511,
 'dest_ip': '192.168.207.4',
 'dest_port': 53,
 'proto': 'UDP',
 'dns': {'type': 'query',
  'id': 43302,
  'rrname': 'safebrowsing.clients.google.com',
  'rrtype': 'A',
  'tx_id': 0}}

##### Normalización

In [11]:
df = pd.json_normalize(clean)

In [12]:
df.head(10)

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,dns.rcode,dns.ttl,dns.rdata
0,2017-07-22T17:33:16.661646-0500,1327836194150542,22269,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680,2001:0500:0001:0000:0000:0000:803f:0235,53,UDP,query,15529,api.wunderground.com,A,0.0,NaN,NaN,NaN
1,2017-07-22T17:33:24.990320-0500,2022925111925872,54352,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051,2001:0500:0003:0000:0000:0000:0000:0042,53,UDP,query,58278,stork79.dropbox.com,A,0.0,NaN,NaN,NaN
2,2017-07-22T17:33:27.379891-0500,578544790391795,54519,dns,150,192.168.205.170,31393,192.168.207.4,53,UDP,query,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN
3,2017-07-22T17:33:27.380146-0500,578544790391795,54520,dns,150,192.168.207.4,53,192.168.205.170,31393,UDP,answer,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,NaN,NaN,NXDOMAIN,NaN,NaN
4,2017-07-22T17:33:27.380146-0500,578544790391795,54520,dns,150,192.168.207.4,53,192.168.205.170,31393,UDP,answer,54724,<root>,SOA,NaN,NXDOMAIN,20864.0,NaN
5,2017-07-22T17:33:36.672785-0500,237919524635665,55496,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,41663,2001:07fd:0000:0000:0000:0000:0000:0001,53,UDP,query,45082,api.wunderground.com,A,0.0,NaN,NaN,NaN
6,2017-07-22T17:33:38.537426-0500,2167545251640146,55687,dns,180,192.168.198.62,35092,192.168.207.4,53,UDP,query,7425,safebrowsing.clients.google.com.home,A,0.0,NaN,NaN,NaN
7,2017-07-22T17:33:38.537676-0500,2167545251640146,55688,dns,180,192.168.207.4,53,192.168.198.62,35092,UDP,answer,7425,safebrowsing.clients.google.com.home,NaN,NaN,NXDOMAIN,NaN,NaN
8,2017-07-22T17:33:38.537676-0500,2167545251640146,55688,dns,180,192.168.207.4,53,192.168.198.62,35092,UDP,answer,7425,<root>,SOA,NaN,NXDOMAIN,20864.0,NaN
9,2017-07-22T17:33:45.672579-0500,541745511744323,56307,dns,120,192.168.202.172,55580,192.168.207.4,53,UDP,query,12854,fxfeeds.mozilla.com,A,0.0,NaN,NaN,NaN


##### Filtro (only DNS tipo A)

In [13]:
clean_df = df[df['dns.rrtype'] == 'A']

In [14]:
len(clean_df)

2849

##### Filtro dominios únicos

In [15]:
clean_df = clean_df.drop_duplicates()

In [16]:
len(clean_df)

2849

In [17]:
clean_df['dns.rrname'].value_counts()

stork79.dropbox.com                312
versioncheck.addons.mozilla.org    120
safebrowsing.clients.google.com     95
mirror.team-cymru.org               84
centos.mirror.facebook.net          84
                                  ... 
www.bigflickrfeed.com                1
gdata.youtube.com                    1
aosnotify.me.com                     1
www.freepbx.org                      1
client-software.real.com             1
Name: dns.rrname, Length: 177, dtype: int64

In [18]:
clean_df.drop_duplicates(subset='dns.rrname', inplace=True)

In [19]:
len(clean_df)

177

In [20]:
clean_df['dns.rrname'].value_counts()

api.wunderground.com                      1
www.cakephp.org                           1
mirror.sanctuaryhost.com                  1
mirror.team-cymru.org                     1
mirror.umoss.org                          1
                                         ..
sourceforge.net                           1
www.freepbx.org                           1
secure.informaction.com.hackerlabs.vpn    1
www.bigflickrfeed.com                     1
client-software.real.com                  1
Name: dns.rrname, Length: 177, dtype: int64

##### Extracción de TLD

In [21]:
from flare.data_science.features import domain_tld_extract, domain_extract

In [22]:
clean_df['tld'] = clean_df.apply(lambda row: domain_tld_extract(row['dns.rrname']), axis=1)

In [23]:
clean_df.head(10)

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,dns.rcode,dns.ttl,dns.rdata,tld
0,2017-07-22T17:33:16.661646-0500,1327836194150542,22269,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680,2001:0500:0001:0000:0000:0000:803f:0235,53,UDP,query,15529,api.wunderground.com,A,0.0,NaN,NaN,NaN,wunderground.com
1,2017-07-22T17:33:24.990320-0500,2022925111925872,54352,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051,2001:0500:0003:0000:0000:0000:0000:0042,53,UDP,query,58278,stork79.dropbox.com,A,0.0,NaN,NaN,NaN,dropbox.com
2,2017-07-22T17:33:27.379891-0500,578544790391795,54519,dns,150,192.168.205.170,31393,192.168.207.4,53,UDP,query,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN,aoltw.net
6,2017-07-22T17:33:38.537426-0500,2167545251640146,55687,dns,180,192.168.198.62,35092,192.168.207.4,53,UDP,query,7425,safebrowsing.clients.google.com.home,A,0.0,NaN,NaN,NaN,safebrowsing.clients.google.com.home
9,2017-07-22T17:33:45.672579-0500,541745511744323,56307,dns,120,192.168.202.172,55580,192.168.207.4,53,UDP,query,12854,fxfeeds.mozilla.com,A,0.0,NaN,NaN,NaN,mozilla.com
12,2017-07-22T17:33:26.875733-0500,1936682168769749,54495,dns,150,192.168.205.188,46250,192.168.207.4,53,UDP,query,11087,www.metasploit.com,A,0.0,NaN,NaN,NaN,metasploit.com
15,2017-07-22T17:33:27.388030-0500,236167177431998,54524,dns,150,192.168.205.170,57181,192.168.207.4,53,UDP,query,38715,aolmtcmxm03.office.aol.com,A,0.0,NaN,NaN,NaN,aol.com
21,2017-07-22T17:33:27.407602-0500,770491173779506,54535,dns,150,192.168.205.170,15001,192.168.207.4,53,UDP,query,38706,aolmtcmxm02.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN,aoltw.net
29,2017-07-22T17:33:27.406254-0500,1289731245028078,54533,dns,150,192.168.205.170,38112,192.168.207.4,53,UDP,query,23609,aolmtcmxm02.office.aol.com,A,0.0,NaN,NaN,NaN,aol.com
35,2017-07-22T17:33:27.378443-0500,862523733034571,54517,dns,150,192.168.205.170,48188,192.168.207.4,53,UDP,query,58554,hpca-tier2.office.aol.com,A,0.0,NaN,NaN,NaN,aol.com


### PARTE 2

### Clasificador de DGA

In [24]:
ndf = pd.read_csv('data/dga_data_small.csv')

In [25]:
ndf.head()

,isDGA,domain,host,subclass
0,dga,tyopcrkqgxcfm,tyopcrkqgxcfm.co.uk,cryptolocker
1,dga,72j5rn1l9mzleo6203v1ogenfl,72j5rn1l9mzleo6203v1ogenfl.org,gameoverdga
2,dga,thenrest,thenrest.net,nivdort
3,dga,15ihbm71utcnfa8dk1mmgoobl9,15ihbm71utcnfa8dk1mmgoobl9.org,gameoverdga
4,dga,x1d6ou7e7kofk60ayhq74x7e,x1d6ou7e7kofk60ayhq74x7e.net,gameoverdga


In [26]:
ndf.drop(['host','subclass'], axis=1, inplace=True)

In [27]:
ndf['isDGA'] = ndf['isDGA'].replace(to_replace='dga', value=1)
ndf['isDGA'] = ndf['isDGA'].replace(to_replace='legit', value=0)

In [28]:
import re

def H_entropy (x):
    # Entropia de Shannon
    prob = [ float(x.count(c)) / len(x) for c in dict.fromkeys(list(x)) ] 
    H = - sum([ p * np.log2(p) for p in prob ]) 
    return H

def posicionPrimerDigito( s ):
    for i, c in enumerate(s):
        if c.isdigit():
            return i + 1
    return 0

def proporcionVocalesConsonantes (x):
    x = x.lower()
    patron_vocales = re.compile('([aeiou])')
    patron_consonantes = re.compile('([b-df-hj-np-tv-z])')
    vocales = re.findall(patron_vocales, x)
    consonantes = re.findall(patron_consonantes, x)
    try:
        proporcion = len(vocales) / len(consonantes)
    except: # Division por cero
        proporcion = 0  
    return proporcion

In [29]:
ndf['longitud'] = ndf['domain'].str.len()
ndf['digitos'] = ndf['domain'].str.count('[0-9]')
ndf['entropia'] = ndf['domain'].apply(H_entropy)
ndf['proporcionVocalesConsonantes'] = ndf['domain'].apply(proporcionVocalesConsonantes)
ndf['posicionPrimerDigito'] = ndf['domain'].apply(posicionPrimerDigito)

ndf.drop(['domain'], axis=1, inplace=True)

In [30]:
print('Final features:', ndf.columns)

Final features: Index(['isDGA', 'longitud', 'digitos', 'entropia',
       'proporcionVocalesConsonantes', 'posicionPrimerDigito'],
      dtype='object')


In [31]:
ndf.head()

,isDGA,longitud,digitos,entropia,proporcionVocalesConsonantes,posicionPrimerDigito
0,1,13,0,3.546594,0.083333,0
1,1,26,10,4.132944,0.333333,1
2,1,8,0,2.500000,0.333333,0
3,1,26,7,4.180833,0.357143,1
4,1,24,9,3.834963,0.666667,2


In [32]:
df_final = ndf

In [33]:
target = df_final['isDGA']
feature_matrix = df_final.drop(['isDGA'], axis=1)

In [34]:
print('Final features:', feature_matrix.columns)

Final features: Index(['longitud', 'digitos', 'entropia', 'proporcionVocalesConsonantes',
       'posicionPrimerDigito'],
      dtype='object')


In [35]:
feature_matrix.head()

,longitud,digitos,entropia,proporcionVocalesConsonantes,posicionPrimerDigito
0,13,0,3.546594,0.083333,0
1,26,10,4.132944,0.333333,1
2,8,0,2.500000,0.333333,0
3,26,7,4.180833,0.357143,1
4,24,9,3.834963,0.666667,2


In [36]:
import sklearn
from sklearn import metrics, model_selection, tree

In [37]:
feature_matrix_train, feature_matrix_test, target_train, target_test = model_selection.train_test_split(feature_matrix, target, test_size=0.25, random_state=31)

In [38]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(feature_matrix_train, target_train)

In [39]:
feature_matrix_train.count()

longitud                        1500
digitos                         1500
entropia                        1500
proporcionVocalesConsonantes    1500
posicionPrimerDigito            1500
dtype: int64

In [40]:
feature_matrix_test.count()

longitud                        500
digitos                         500
entropia                        500
proporcionVocalesConsonantes    500
posicionPrimerDigito            500
dtype: int64

##### Testing with new data

In [41]:
clean_df.drop(clean_df.columns.difference(['tld','dns.rrname']), 1, inplace=True)

C:\Users\Gerardo\AppData\Local\Temp\ipykernel_12956\625639539.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  clean_df.drop(clean_df.columns.difference(['tld','dns.rrname']), 1, inplace=True)


In [42]:
host_df = clean_df.copy()

In [43]:
host_df['domain'] = host_df.apply(lambda row: domain_extract(row['dns.rrname']), axis=1)

In [44]:
host_df.head(50)

,dns.rrname,tld,domain
0,api.wunderground.com,wunderground.com,wunderground
1,stork79.dropbox.com,dropbox.com,dropbox
2,hpca-tier2.office.aol.com.ad.aol.aoltw.net,aoltw.net,aoltw
6,safebrowsing.clients.google.com.home,safebrowsing.clients.google.com.home,NaN
9,fxfeeds.mozilla.com,mozilla.com,mozilla
12,www.metasploit.com,metasploit.com,metasploit
15,aolmtcmxm03.office.aol.com,aol.com,aol
21,aolmtcmxm02.office.aol.com.ad.aol.aoltw.net,aoltw.net,aoltw
29,aolmtcmxm02.office.aol.com,aol.com,aol
35,hpca-tier2.office.aol.com,aol.com,aol


In [45]:
host_df.dropna(inplace=True)

In [46]:
host_df.head(20)

,dns.rrname,tld,domain
0,api.wunderground.com,wunderground.com,wunderground
1,stork79.dropbox.com,dropbox.com,dropbox
2,hpca-tier2.office.aol.com.ad.aol.aoltw.net,aoltw.net,aoltw
9,fxfeeds.mozilla.com,mozilla.com,mozilla
12,www.metasploit.com,metasploit.com,metasploit
15,aolmtcmxm03.office.aol.com,aol.com,aol
21,aolmtcmxm02.office.aol.com.ad.aol.aoltw.net,aoltw.net,aoltw
29,aolmtcmxm02.office.aol.com,aol.com,aol
35,hpca-tier2.office.aol.com,aol.com,aol
39,aolmtcmxm03.office.aol.com.ad.aol.aoltw.net,aoltw.net,aoltw


In [47]:
host_df['longitud'] = host_df['domain'].str.len()
host_df['digitos'] = host_df['domain'].str.count('[0-9]')
host_df['entropia'] = host_df['domain'].apply(H_entropy)
host_df['proporcionVocalesConsonantes'] = host_df['domain'].apply(proporcionVocalesConsonantes)
host_df['posicionPrimerDigito'] = host_df['domain'].apply(posicionPrimerDigito)

In [48]:
domains = host_df['tld']
host_df.drop(['domain', 'tld', 'dns.rrname'], axis=1, inplace=True)

In [49]:
print('Final features:', host_df.columns)

Final features: Index(['longitud', 'digitos', 'entropia', 'proporcionVocalesConsonantes',
       'posicionPrimerDigito'],
      dtype='object')


In [50]:
host_df.head()

,longitud,digitos,entropia,proporcionVocalesConsonantes,posicionPrimerDigito
0,12,0,2.918296,0.500000,0
1,7,0,2.521641,0.400000,0
2,5,0,2.321928,0.666667,0
9,7,0,2.521641,0.750000,0
12,10,0,3.121928,0.666667,0


In [51]:
target_pred = clf.predict(host_df)

In [52]:
pred_df = host_df

In [53]:
pred_df['isDGA'] = target_pred

In [54]:
pred_df['tld'] = domains

In [55]:
pred_df[pred_df['isDGA'] == 1]

,longitud,digitos,entropia,proporcionVocalesConsonantes,posicionPrimerDigito,isDGA,tld
175,12,0,3.084963,0.714286,0,1,informaction.com
648,15,0,3.506891,0.400000,0,1,backtrack-linux.org
1024,13,0,3.238901,0.444444,0,1,bigflickrfeed.com
2445,13,0,3.238901,0.444444,0,1,sanctuaryhost.com
9856,16,0,3.250000,0.142857,0,1,vtlfccmfxlkgifuf.com


### PARTE 3

##### Dentro de Umbrella

In [56]:
from flare.tools.umbrella import Umbrella

In [57]:
umbrellaCisco = Umbrella(limit=1000000)

[+] Updated Umbrella Top 1 Million list...


In [58]:
pred_df['isInUmbrella'] = pred_df.apply(lambda row: umbrellaCisco.domain_in_umbrella(row['tld']), axis=1)

In [59]:
pred_df[pred_df['isDGA'] == 1]

,longitud,digitos,entropia,proporcionVocalesConsonantes,posicionPrimerDigito,isDGA,tld,isInUmbrella
175,12,0,3.084963,0.714286,0,1,informaction.com,True
648,15,0,3.506891,0.400000,0,1,backtrack-linux.org,False
1024,13,0,3.238901,0.444444,0,1,bigflickrfeed.com,False
2445,13,0,3.238901,0.444444,0,1,sanctuaryhost.com,False
9856,16,0,3.250000,0.142857,0,1,vtlfccmfxlkgifuf.com,False


##### WhoIs Lookup

In [60]:
import whois

In [75]:
def get_creation_date(row):
    response = ''
    print(row['tld'])
    try:
        response = whois.whois(row['tld']).creation_date
    except:
        print('Domain info not found')
        
    return response

In [77]:
dga_df = pred_df.copy()

In [76]:
pred_df['dcreation_date'] = pred_df.apply(lambda row: get_creation_date(row), axis=1)

wunderground.com
dropbox.com
aoltw.net
mozilla.com
metasploit.com
aol.com
aoltw.net
aol.com
aol.com
Error trying to connect to socket: closing socket
aoltw.net
aol.com
google.com
stayonline.net
aoltw.net
aol.com
aol.com
informaction.com
vmware.com
mozilla.com
windows.com
vmware.com
ntkrnlpa.info
portswigger.net
offensive-security.com
Error trying to connect to socket: closing socket
stayonline.net
stopbadware.org
Domain info not found
aoltw.net
arrancar.org
sql-ledger.org
backtrack-linux.org
Error trying to connect to socket: closing socket
stayonline.net
mozilla.com
theanime.cn
Domain info not found
theanime.cn
Domain info not found
aoltw.net
aoltw.net
aoltw.net
stayonline.net
aoltw.net
aol.com
aoltw.net
phpmyadmin.net
Error trying to connect to socket: closing socket
google.com
google.com
microsoft.com
aoltw.net
comcast.net
google.com
aoltw.net
ubuntu.com
mozilla.com
alexa.com
postgresql.org
Error trying to connect to socket: closing socket
sourceforge.net
freepbx.org
bigflickrfeed.c

In [79]:
pred_df[pred_df['isDGA'] == 1]

,longitud,digitos,entropia,proporcionVocalesConsonantes,posicionPrimerDigito,isDGA,tld,isInUmbrella,dcreation_date
175,12,0,3.084963,0.714286,0,1,informaction.com,True,1998-11-25 05:00:00
648,15,0,3.506891,0.400000,0,1,backtrack-linux.org,False,2009-04-29 00:26:42
1024,13,0,3.238901,0.444444,0,1,bigflickrfeed.com,False,
2445,13,0,3.238901,0.444444,0,1,sanctuaryhost.com,False,"[2004-07-13 15:17:11, 2004-07-13 15:17:00]"
9856,16,0,3.250000,0.142857,0,1,vtlfccmfxlkgifuf.com,False,


**¿Qué dominios es más probable que sean maliciosos (basado en la fecha, el resultado del modelo y el Cisco Umbrella)?**

Los dominios que seguramente son maliciosos son aquellos que el modelo marcó como DGA, que no se encuentran dentro del Cisco Umbrella y los que no tienen datos sobre la fecha de creación. Los dominios con fecha de creación antigua también pueden ser sospechosos, y en conjunto con el resultado del modelo puede demostrar ser malicioso.

Dominios maliciosos:
- bigflickrfeed.com
- vtlfccmfxlkgifuf.com